In [0]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls -lha kaggle.json
!kaggle competitions download -c asap-aes

Saving kaggle.json to kaggle.json
-rw-r--r-- 1 root root 65 Jul 17 19:56 kaggle.json
 58% 9.00M/15.6M [00:00<00:00, 16.4MB/s]
100% 15.6M/15.6M [00:00<00:00, 26.5MB/s]
 26% 5.00M/19.0M [00:00<00:00, 28.2MB/s]
100% 19.0M/19.0M [00:00<00:00, 63.3MB/s]
 78% 5.00M/6.39M [00:00<00:00, 29.4MB/s]
100% 6.39M/6.39M [00:00<00:00, 31.3MB/s]
 99% 5.00M/5.05M [00:00<00:00, 31.5MB/s]
100% 5.05M/5.05M [00:00<00:00, 24.7MB/s]
 82% 5.00M/6.12M [00:00<00:00, 30.1MB/s]
100% 6.12M/6.12M [00:00<00:00, 29.9MB/s]
  0% 0.00/2.06M [00:00<?, ?B/s]
100% 2.06M/2.06M [00:00<00:00, 65.8MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 11.9MB/s]
  0% 0.00/14.9k [00:00<?, ?B/s]
100% 14.9k/14.9k [00:00<00:00, 16.4MB/s]
  0% 0.00/41.4k [00:00<?, ?B/s]
100% 41.4k/41.4k [00:00<00:00, 42.1MB/s]
  0% 0.00/88.5k [00:00<?, ?B/s]
100% 88.5k/88.5k [00:00<00:00, 76.3MB/s]
  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 65.3MB/s]
 75% 41.0M/55.0M [00:01<00:00, 28.4MB/s]
100% 55.0M/55.0M [00:01<00:00

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np 
import pandas as pd 
from pathlib import Path
from fastai.text import *

In [153]:
train = pd.read_csv('training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
train=train[train['essay_set']==8]

train['trait1'] = (train['rater1_trait1'] + train['rater2_trait1'])
train['trait2'] = (train['rater1_trait2'] + train['rater2_trait2'])
train['trait3'] = (train['rater1_trait3'] + train['rater2_trait3'])
train['trait4'] = (train['rater1_trait4'] + train['rater2_trait4'])
train['trait5'] = (train['rater1_trait5'] + train['rater2_trait5'])
train['trait6'] = (train['rater1_trait6'] + train['rater2_trait6'])

alltrain = train[['essay','trait1','trait2','trait3','trait4','trait5','trait6',]]
train1 = train[['essay','trait6']]

print(train1.shape)
train1.head()

(723, 2)


,essay,trait6
12253,A long time ago when I was in third grade I h...,6.0
12254,Softball has to be one of the single most gre...,9.0
12255,"Some people like making people laugh, I love ...",7.0
12256,"""LAUGHTER"" @CAPS1 I hang out with my friends...",6.0
12257,Well ima tell a story about the time i got @CA...,5.0


In [0]:
train1=train1.sample(frac=1)

In [114]:
train=train1.iloc[0:578]
test=train1.iloc[578:723]

print(train.shape,test.shape)

(578, 2) (145, 2)


In [0]:
path=Path('/content/gdrive/My Drive')

data_lm = TextLMDataBunch.from_df(path=path,
 train_df=train,
 valid_df=test, 
 label_cols='trait6', 
 text_cols='essay')

In [0]:
lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
lm.save_encoder('encoder6.pth')

In [0]:
data_clas=TextClasDataBunch.from_df(path=path,train_df=train, valid_df=test, vocab=data_lm.train_ds.vocab, 
                                    label_cols='trait6', text_cols='essay')

In [0]:
clas = text_classifier_learner(data_clas,AWD_LSTM,drop_mult=0.3)

In [0]:
clas.load_encoder('encoder6.pth')

In [0]:
#clas.lr_find()
#clas.recorder.plot(suggestion=True)

In [121]:
clas.fit_one_cycle(5, 1e-1)

epoch,train_loss,valid_loss,time
0,47.447849,13.740945,00:19
1,26.525360,5.099241,00:18
2,17.130320,1.777853,00:18
3,12.016137,1.165692,00:19
4,9.013661,1.144729,00:19


In [0]:
clas.export('t6model.pkl')

In [0]:
 t1=load_learner('/content/gdrive/My Drive/',file='t1model.pkl')

In [138]:
clas.show_results()

text,target,prediction
"xxbos xxmaj laughter is the best medicine xxmaj many people say laughter is the best medicine . i for one , agree . xxmaj the reason why is that in many points of xxup @caps27 life , laughter has helped me , or others , feel a whole lot better . xxup @caps27 mother recently got xxunk on her shoulder . xxup @caps4 had medicine xxup @caps2 all xxup @caps34",8.0,[6.599689]
"xxbos xxmaj xxunk "" xxup @caps1 the heck is her problem , "" is xxup @caps1 i thought as i glanced over and xxup @caps7 gave me xxup @caps15 nasty look . xxup @caps27 obviously thought the new girl was weird . xxup @caps7 xxup @caps24 had xxup @caps15 xxunk look in her eyes . i observed the way xxup @caps7 glared at xxup @caps27 in the room , and",8.0,[7.004791]
"xxbos xxunk 's get out of here before they wake up xxup @caps2 . "" xxup @person2 said , standing up with her blanket xxunk around her . i did the same , xxunk grass of my pillow . xxmaj disappointment was rushing over me , i actually wanted to see the meteor shower . "" xxup @caps3 coming xxup @person5 ? "" i asked , looking at my old",8.0,[6.98792]
"xxbos xxmaj it is often said that laughter can bring people together . i hold that knowledge to be true . xxmaj we 've all laughed at one point in our lives , whether it was in a group of people , or just laughing , and having a xxup @caps9 time . xxmaj reality is we are all connected to each other in one very simple way , we",8.0,[6.833342]
"xxbos xxup @caps11 was xxup @date1 , hot and dry had n't seen any xxup @caps26 for a few miles . xxmaj we 'd been walking for a good six hours now xxup @caps18 xxunk on left over xxunk xxunk . xxup @person3 had n't said much today still pissed off about dropping the can - xxunk , i would 've laughed if the xxunk of our lives were n't",7.0,[6.920996]


In [137]:
train.head()

,essay,trait6
12302,@ORGANIZATION1 is one of the many joys I love ...,7.0
12731,Some people say that laughter is the best med...,8.0
12737,My friends and I have a relationship that can...,7.0
12678,Laughing at @CAPS1 bell. @CAPS11 best friend ...,7.0
12943,When I think of laughter you think of memorie...,7.0


In [152]:
ex=train['essay'].iloc[0]
y=clas.predict(ex)[0]

import math

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

print(y)
print(sigmoid(y.data[0]))

[6.868919]
0.9989614797675161


TypeError: ignored